# Segmenting and Clustering Neighborhoods in Toronto


#### Abstract

   This project utilizes Foursquare API to retrieve details on venues located in Toronto neighborhoods and applies the K-Means alogrithm to view similarities and dissimilarities between the neighborhoods based on the venue category feature retrieved. The dataset is derived from the "List of Postal Codes" wikipedia page (https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) for Toronto, with location (latitude and longitude) data added for plotting maps.

#### Method

   From the lecture, K-Means is defined as a partitioning clustering algorithm and segments data into clustsers without any predefined internal structure. Objects of one cluster greatly differ from the that of another cluster, but are very similar to each other in their own cluster. Functions such as the "get_category_type" and "getNearbyVenues" were taken from the lecture among other raw code used to process the foursquare data.

  The Foursquare API pulls all venues located in a 500m radius of the collection of neighborhoods or borough coordinates (latitude and longitude based on postal code). The approximately 1700 venues (displayed by their category) are each assigned a value of 1 through one hot encoding. This dataframe is then grouped by borough coordinates. The venues or venue categories of the same type are added, and then divided by the total number of venues for that neighborhood, i.e. the "mean" value for each venue category is obtained for each unique borough latitude and longitude.

#### Result

   The number of clusters is set to 5 with iterations set to 10. After running K-Means multiple times, the algorithm consistently assigns majority of the neighborhoods to a single cluster, while the other 4 are slightly unique and most often present either in central or downtown Toronto.

#### Conclusion

   The cluster with the largest number of objects had "coffee shop", "cafe", a restaurant of a specific cuisine among "bar", "park", "pub" as the venue categories that appeared the most or in the Top 5. Most of the objects in that cluster had one or a few unrelated venue categories, whereas the other clusters had a more unique combination of venue categories with some of the more frequent venue categories of the large cluster appearing at the bottom of the Top 10 list or having higher mean frequencies than that of objects within the largest cluster. On the contrary, the neighborhoods King and Spadina, CN Tower, Bathurst Quay, Railway Lands, Island airport, Harbourfront West, South Niagara all clubbed under a lat and long assigned to Downtown Toronto have venue categories exclusively Airport related. This is saying that the mean frequency of airport related venue categories are close to the mean frequency of other venues categories of the objects within that cluster despite the uniqueness of the venue categories. However, there were instances where this particular neighborhood object was not part of the largest cluster and on its own, which shows how K-Means is inconsistent due to the randomness of the initial centroid starting points.

## Building the dataframe

###### Importing packages to scrape wiki page for neighborhood data

In [1]:
import bs4 as BeautifulSoup
import pandas as pd
import numpy as np
import requests
import json
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.colors as colors
print("Packages are imported!")

Packages are imported!


###### Initiating BeautifulSoup parser on URL stored as file object 

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_doc = requests.get(url)
file = open('data.html','wb').write(html_doc.content)
print("File has been created",", ","space = ",file,"bytes")

File has been created ,  space =  80694 bytes


In [3]:
# Opening file and reading HTML table data
read_file = open('data.html','r', encoding="utf8")
data = read_file.read()
print(data)

# Initiating html parser using BeautifulSoup package
soup = BeautifulSoup.BeautifulSoup(data,'html')

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":876823784,"wgRevisionId":876823784,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","w

###### Gathering postal code, neighborhood, and borough data from html file

In [4]:
counter = 1
postal_code = []
borough = []
neighborhood = []
while counter <= 289:
    table = soup.find_all('table')[0].tbody.find_all('tr')[counter]
    pc = table.find_all('td')[0].text
    br = table.find_all('td')[1].text
    nh = table.find_all('td')[2].text.rstrip()
    postal_code.append(pc)
    borough.append(br)
    neighborhood.append(nh)
    counter = counter + 1

###### Storing gathered information to a new dataframe

In [5]:
df_table = pd.DataFrame()
df_table['PostalCode'] = postal_code
df_table['Borough'] = borough
df_table['Neighborhood'] = neighborhood
df_table.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


###### Removing rows wherein boroughs are "Not assigned"

In [6]:
df_table = df_table[df_table.Borough != "Not assigned"]
df_table.head(10)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


###### Replacing neighborhoods that are "Not assigned" with their respective borough label 

In [7]:
df_table['Neighborhood'] = df_table['Neighborhood'].replace('Not assigned', df_table['Borough'])
df_table.head(10)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


###### Merging neighborhoods of the same postal code

In [8]:
df_table = df_table.groupby('PostalCode').agg(lambda x: ", ".join(list(set(x.tolist()))))
df_table.head()

,Borough,Neighborhood
PostalCode,,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Port Union, Highland Creek, Rouge Hill"
M1E,Scarborough,"Morningside, West Hill, Guildwood"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


###### Resetting the dataframe index (starting from 0)

In [9]:
df_table = df_table.reset_index()
df_table.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Port Union, Highland Creek, Rouge Hill"
2,M1E,Scarborough,"Morningside, West Hill, Guildwood"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


###### Calling the dimensions of the dataframe

In [10]:
df_table.shape

(103, 3)

###### Storing location data to a dataframe

In [11]:
df_GeoCoords = pd.read_csv('http://cocl.us/Geospatial_data')
df_GeoCoords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


###### Merging neighborhood and location dataframes

In [12]:
df_SegClusTorontoData = pd.concat([df_table, df_GeoCoords], axis=1)
df_SegClusTorontoData = df_SegClusTorontoData.drop(columns=['Postal Code'])
df_SegClusTorontoData.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Port Union, Highland Creek, Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Morningside, West Hill, Guildwood",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Import Folium package for mapping

In [13]:
#import sys
#!{sys.executable} -m pip install folium

In [14]:
import folium
print("Folium package successfully imported")

Folium package successfully imported


## Initial map of neighborhoods in Toronto

In [15]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=10)

# add markers to map
for pstlcode, lat, lng, borough, neighborhood in zip(df_SegClusTorontoData['PostalCode'], df_SegClusTorontoData['Latitude'], df_SegClusTorontoData['Longitude'], df_SegClusTorontoData['Borough'], df_SegClusTorontoData['Neighborhood']):
    label = '{}, {}, {}'.format(neighborhood, borough, pstlcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)

    
map_toronto

## Filtering for neighborhoods only in Toronto

###### Filtering for boroughs containing "Toronto"

In [16]:
df_SegClusTorontoData = df_SegClusTorontoData[df_SegClusTorontoData['Borough'].str.contains('Toronto')]
df_SegClusTorontoData

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
49,M4V,Central Toronto,"Deer Park, Summerhill West, Rathnelly, South H...",43.686412,-79.400049


###### Resetting the index (starting from 0) 

In [17]:
df_SegClusTorontoData = df_SegClusTorontoData.reset_index()
df_SegClusTorontoData.head()

,index,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,37,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,43,M4M,East Toronto,Studio District,43.659526,-79.340923
4,44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [18]:
del df_SegClusTorontoData['index']
df_SegClusTorontoData.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


## Pulling all venues using Foursquare API

###### Defining Foursquare API credentials

In [19]:
CLIENT_ID = ''
CLIENT_SECRET = ''
VERSION = '20180605'

print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:


###### Applying Foursquare API on the first neighborhood

In [20]:
df_SegClusTorontoData.loc[0,'Neighborhood']

'The Beaches'

In [21]:
neighborhood_latitude = df_SegClusTorontoData.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_SegClusTorontoData.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_SegClusTorontoData.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


In [22]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=SITK0Y5ZZVIMT511CTKGYRXAMRQLZ2NQLJHASDU2JHOFCLEL&client_secret=JAZC3A2WWMOWJXCXBVCWCQEVVFFPUN3HFC5WBA1M2QVBRFR1&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

In [23]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c54acf24434b97443d6f754'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b8daea1f964a520480833e3',
       'name': 'Grover Pub and Grub',
       'location': {'address': '676 Kingston Rd.',
        'crossStreet': 'at Main St.',
        'lat': 43.679181434941015,
        'lng': -79.29721535878515,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.679181434941015,
          'lng': -79.29721535878515}],
    

###### Defining a function that extracts the category of the venue

In [24]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

###### Extracting the venues associated with the neighborhood by applying the "get_category_type" function

In [25]:
venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Grover Pub and Grub,Pub,43.679181,-79.297215
1,Starbucks,Coffee Shop,43.678798,-79.298045
2,Guru Raghavendra Ji,Astrologer,43.680187,-79.292337
3,Upper Beaches,Neighborhood,43.680563,-79.292869


###### Showing the number of venues that were returned

In [26]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


## Exploring the neighborhoods

###### Defining a function to return the most common venue categories in each neighborhood

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
toronto_venues = getNearbyVenues(names=df_SegClusTorontoData['Neighborhood'],
                                   latitudes=df_SegClusTorontoData['Latitude'],
                                   longitudes=df_SegClusTorontoData['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Summerhill West, Rathnelly, South Hill, Forest Hill SE
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, King, Adelaide
Harbourfront East, Toronto Islands, Union Station
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
North Midtown, The Annex, Yorkville
Harbord, University of Toronto
Kensington Market, Grange Park, Chinatown
CN Tower, Railway Lands, Harbourfront West, Island airport, South Niagara, Bathurst Quay, King and Spadina
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Exhibition Place, Brockton, Parkdale Village
The Junction So

###### Viewing the total number of venues called into the dataframe along with a sample of the called dataframe

In [29]:
print(toronto_venues.shape)
toronto_venues

(1700, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
1,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
2,The Beaches,43.676357,-79.293031,Guru Raghavendra Ji,43.680187,-79.292337,Astrologer
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
5,"The Danforth West, Riverdale",43.679557,-79.352188,Dolce Gelato,43.677773,-79.351187,Ice Cream Shop
6,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
7,"The Danforth West, Riverdale",43.679557,-79.352188,Messini Authentic Gyros,43.677827,-79.350569,Greek Restaurant
8,"The Danforth West, Riverdale",43.679557,-79.352188,Mezes,43.677962,-79.350196,Greek Restaurant
9,"The Danforth West, Riverdale",43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant


## Using the "venue category" feature to group the neighborhoods into clusters

###### Using one-hot encoding to convert categorical data into useable data for the K-means clustering algorithm

In [30]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
toronto_onehot.shape

(1700, 239)

In [32]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
1,Business Reply Mail Processing Centre 969 Eastern,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,"CN Tower, Railway Lands, Harbourfront West, Is...",0.000000,0.000000,0.000000,0.076923,0.076923,0.076923,0.153846,0.153846,0.153846,...,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,Central Bay Street,0.012195,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.00000,0.00,0.012195,0.000000,0.000000,0.012195,0.000000,0.00
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,Church and Wellesley,0.011628,0.011628,0.011628,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.00000,0.00,0.000000,0.011628,0.011628,0.000000,0.011628,0.00
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.010000,0.000000,0.00
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.027778,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00


In [33]:
toronto_grouped.shape

(38, 239)

###### Listing Top 5 venues based on their frequency

In [34]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2          Restaurant  0.05
3  Seafood Restaurant  0.04
4              Bakery  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.06
2       Auto Workshop  0.06
3       Garden Center  0.06
4              Garden  0.06


----CN Tower, Railway Lands, Harbourfront West, Island airport, South Niagara, Bathurst Quay, King and Spadina----
              venue  freq
0    Airport Lounge  0.15
1   Airport Service  0.15
2  Airport Terminal  0.15
3          Boutique  0.08
4  Sculpture Garden  0.08


----Cabbagetown, St. James Town----
                venue  freq
0         Coffee Shop  0.11
1          Restaurant  0.09
2              Bakery  0.04
3  Italian Restaurant  0.04
4                 Pub  0.04


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.16
1                Caf

###### Defining a function that returns the venues with the highest frequencies for a given neighborhood based on "num_top_venues" input

In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

###### Forming a dataframe with Top 10 most freqeunt venues using the "return_most_common_venues" function 

In [36]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Pub,Cheese Shop,Farmers Market,Bakery,Café,Beer Bar,Seafood Restaurant
1,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Garden,Farmers Market,Spa,Fast Food Restaurant,Brewery,Restaurant,Burrito Place,Recording Studio
2,"CN Tower, Railway Lands, Harbourfront West, Is...",Airport Lounge,Airport Service,Airport Terminal,Boutique,Airport,Airport Food Court,Airport Gate,Harbor / Marina,Sculpture Garden,Boat or Ferry
3,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Park,Bakery,Italian Restaurant,Pub,Café,Pizza Place,Pharmacy,Sandwich Place
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Bar,Burger Joint,Middle Eastern Restaurant,Thai Restaurant,Salad Place,Bubble Tea Shop,Spa


## Utilizing the k-means clustering algorithm to cluster neighborhoods in Toronto based on the "venue category" feature

In [37]:
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(init = 'k-means++', n_init=10, max_iter=300, n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

print(kmeans.labels_) # each entire row


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 4 0 1 0 0 0 0 2 3 0 0 0 0 0 0 0 0
 0]


######  Merging "toronto_grouped" with "toronto_data" to add latitude/longitude for each neighborhood after adding cluster labels

In [38]:
toronto_merged = df_SegClusTorontoData

toronto_merged['Cluster Labels'] = kmeans.labels_

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Pub,Astrologer,Women's Store,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Yoga Studio,Brewery,Bakery,Spa,Juice Bar
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Park,Pub,Ice Cream Shop,Board Shop,Sandwich Place,Brewery,Fast Food Restaurant,Fish & Chips Shop,Burger Joint,Burrito Place
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Bar,Fish Market,Juice Bar,Coworking Space,Latin American Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Gym / Fitness Center,Park,Swim School,Bus Line,Concert Hall,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


### Toronto neighborhoods have been segmented and clustered based on the most frequented venues  

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Coffee Shop,Pub,Astrologer,Women's Store,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
1,East Toronto,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Yoga Studio,Brewery,Bakery,Spa,Juice Bar
2,East Toronto,0,Park,Pub,Ice Cream Shop,Board Shop,Sandwich Place,Brewery,Fast Food Restaurant,Fish & Chips Shop,Burger Joint,Burrito Place
3,East Toronto,0,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Bar,Fish Market,Juice Bar,Coworking Space,Latin American Restaurant
4,Central Toronto,0,Gym / Fitness Center,Park,Swim School,Bus Line,Concert Hall,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
5,Central Toronto,0,Gym,Clothing Store,Burger Joint,Sandwich Place,Food & Drink Shop,Hotel,Breakfast Spot,Park,Grocery Store,Dumpling Restaurant
6,Central Toronto,0,Coffee Shop,Sporting Goods Shop,Yoga Studio,Spa,Dessert Shop,Mexican Restaurant,Salon / Barbershop,Sandwich Place,Fast Food Restaurant,Chinese Restaurant
7,Central Toronto,0,Pizza Place,Sandwich Place,Dessert Shop,Café,Sushi Restaurant,Italian Restaurant,Seafood Restaurant,Coffee Shop,Toy / Game Store,Greek Restaurant
8,Central Toronto,0,Restaurant,Playground,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
9,Central Toronto,0,Pub,Coffee Shop,American Restaurant,Bagel Shop,Sports Bar,Restaurant,Supermarket,Sushi Restaurant,Fried Chicken Joint,Light Rail Station


In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,1,Health & Beauty Service,Home Service,Garden,Concert Hall,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,2,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Restaurant,Tea Room,Bar,Ramen Restaurant,Diner
27,Downtown Toronto,2,Airport Lounge,Airport Service,Airport Terminal,Boutique,Airport,Airport Food Court,Airport Gate,Harbor / Marina,Sculpture Garden,Boat or Ferry


In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Downtown Toronto,3,Coffee Shop,Restaurant,Café,Cocktail Bar,Pub,Seafood Restaurant,Hotel,Beer Bar,Italian Restaurant,Japanese Restaurant


In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Downtown Toronto,4,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gastropub,Italian Restaurant,Gym,Deli / Bodega,Japanese Restaurant


## Using the Folium library to visualize the clusters

In [44]:
# create map
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## End